<p style="font-size:44px; text-align:center; font-weight:bold">
    Fine-tuning for poems and songs
</p>

<p style="font-size:20px">
    Initialize for consitency
</p>

In [26]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

device: cuda
random seed: 1234


<p style="font-size:35px; text-align:center; font-weight:bold">
    Pre-tuning setup
</p>

<p style="font-size:20px; font-weight:bold">
    Import tuned roberta model
</p>

In [27]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

name = './roberta-base-sentiments'
tokenizer = RobertaTokenizer.from_pretrained(name)
model = RobertaForSequenceClassification.from_pretrained(name)
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

<p style="font-size:20px; font-weight:bold">
    Import and adjust datasets used for tuning and testing
</p>

In [28]:
"""
Importing the datasets
"""

from datasets import load_dataset, Dataset

# Songs
kg_songs = load_dataset("csv", data_files="../datasets/songs.csv")
kg_songs = kg_songs["train"].to_pandas()

# Poems
kg_poems = load_dataset("csv", data_files="../datasets/final_df_emotions(remove-bias).csv")
kg_poems = kg_poems["train"].to_pandas()

md_perc = pd.read_excel("../datasets/PERC_mendelly.xlsx")

In [29]:
"""
Adjusting the datasets
"""

# Rename columns for consistency
kg_songs = kg_songs.rename(columns={"mood": "label", "lyrics": "text"})
kg_poems = kg_poems.rename(columns={"poem content": "text"})
md_perc = md_perc.rename(columns={"Emotion": "label", "Poem": "text"})

labels = ["sad", "joy", "love", "anger", "fear", "surprise"]
label_map = {
    "sad": 0,
    "sadness": 0,
    "joy": 1,
    "happy": 1,
    "love": 2,
    "romantic": 2,
    "anger": 3,
    "angry": 3,
    "fear": 4,
    "surprise": 5,
    "excited": 5
}

# Adjust labels to match the model's expected input
def adjust_labels(df, label_col, label_map):
    """
    This will drop rows that do not have labels in the label_map
    """
    df[label_col] = df[label_col].map(label_map)
    return df[df[label_col].isin([0,1,2,3,4,5])].reset_index(drop=True)

kg_songs = adjust_labels(kg_songs, "label", label_map)
kg_poems = adjust_labels(kg_poems, "label", label_map)
md_perc = adjust_labels(md_perc, "label", label_map)

<p style="font-size:20px; font-weight:bold">
    Seperate datasets into further training and testing datasets
</p>

In [30]:
kg_songs_ds = Dataset.from_pandas(kg_songs)
kg_poems_ds = Dataset.from_pandas(kg_poems)
md_perc_ds = Dataset.from_pandas(md_perc)

In [31]:
print(kg_songs_ds)

Dataset({
    features: ['song_name', 'artist', 'text', 'label'],
    num_rows: 498
})


In [18]:
from transformers import Trainer
def tokenize(dataset):
    return tokenizer(dataset['text'], truncation=True, padding='max_length')

test1 = kg_songs_ds.map(
    tokenize, batched=True,
    remove_columns=['song_name', 'artist'],
)

test2 = kg_poems_ds.map(
    tokenize, batched=True,
    remove_columns=['pred', 'score', 'anger', 'fear', 'joy', 'disgust', 'sadness', 'surprise', 'neutral'],
)

test3 = md_perc_ds.map(
    tokenize, batched=True,
)

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

Map:   0%|          | 0/536 [00:00<?, ? examples/s]

In [19]:
from datasets import Value

test1 = test1.cast_column("label", Value("int64"))
test2 = test2.cast_column("label", Value("int64"))
test3 = test3.cast_column("label", Value("int64"))
trainer = Trainer(model=model)
output1 = trainer.predict(test1)
output2 = trainer.predict(test2)
output3 = trainer.predict(test3)

Casting the dataset:   0%|          | 0/498 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/374 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/536 [00:00<?, ? examples/s]

In [23]:
from sklearn.metrics import classification_report

labels = ["sad", "joy", "love", "anger", "fear", "surprise"]
y_true = output1.label_ids
y_pred = np.argmax(output1.predictions, axis=-1)
target_names = labels
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         sad       0.13      0.02      0.04        95
         joy       0.21      0.42      0.28       106
        love       0.29      0.03      0.05        73
       anger       0.24      0.50      0.32       113
        fear       0.00      0.00      0.00         0
    surprise       0.00      0.00      0.00       111

    accuracy                           0.21       498
   macro avg       0.14      0.16      0.12       498
weighted avg       0.17      0.21      0.15       498



/home/csc396_7/miniconda3/envs/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/csc396_7/miniconda3/envs/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/csc396_7/miniconda3/envs/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", res

In [24]:
y_true = output2.label_ids
y_pred = np.argmax(output2.predictions, axis=-1)
target_names = labels
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         sad       0.45      0.29      0.35       143
         joy       0.13      0.79      0.22        24
        love       0.00      0.00      0.00         0
       anger       0.33      0.37      0.35        67
        fear       0.73      0.17      0.28       128
    surprise       0.50      0.25      0.33        12

    accuracy                           0.29       374
   macro avg       0.36      0.31      0.26       374
weighted avg       0.51      0.29      0.32       374



/home/csc396_7/miniconda3/envs/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/csc396_7/miniconda3/envs/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/csc396_7/miniconda3/envs/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", res

In [25]:
y_true = output3.label_ids
y_pred = np.argmax(output3.predictions, axis=-1)
target_names = labels
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         sad       0.36      0.37      0.36       154
         joy       0.31      0.45      0.37       128
        love       0.70      0.20      0.31       161
       anger       0.14      0.24      0.17        46
        fear       0.12      0.19      0.15        31
    surprise       0.25      0.19      0.21        16

    accuracy                           0.31       536
   macro avg       0.31      0.27      0.26       536
weighted avg       0.41      0.31      0.31       536

